# Data Preprocessing

### Setup and Load Data

In [203]:
import pandas as pd

df = pd.read_csv('data/layoffs_panel_final.csv')
df['Date'] = pd.to_datetime(df['Date'])

print(f"Dataset shape: {df.shape}")
print(f"Companies: {df['Company'].nunique()}")
print(f"Date range: {df['Date'].min().date()} to {df['Date'].max().date()}")
print(f"\nColumns ({len(df.columns)}):")
print(df.columns.tolist())

print(f"\nTarget variable distribution:")
print(df['Layoff_Event_Binary'].value_counts())

print(f"\nMissing values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing[missing > 0],
    'Missing_Percentage': missing_pct[missing > 0]
}).sort_values('Missing_Count', ascending=False)
missing_df_filtered = missing_df[~missing_df.index.str.contains('_lag2|_lag4|_lag5', regex=True)]

display(missing_df_filtered)

Dataset shape: (13936, 79)
Companies: 268
Date range: 2020-03-01 to 2024-06-01

Columns (79):
['Company', 'Date', 'Layoff_Event_Count', 'Total_Laid_Off_Sum', 'Avg_Layoff_Percentage', 'Latest_Industry', 'Latest_Country', 'Latest_Stage', 'Latest_Funds_Raised', 'Layoff_Event_Binary', 'unemployment_rate_lag1', 'cpi_lag1', 'consumer_confidence_lag1', 'gdp_growth_rate_lag1', 'fed_funds_rate_lag1', 'avg_jobless_claims_lag1', 'sp500_index_lag1', 'inflation_rate_yoy_lag1', 'sp500_change_6mo_lag1', 'company_sec', 'fiscal_year', 'fiscal_period', 'form', 'cash_lag1', 'cost_of_revenue_lag1', 'gross_profit_lag1', 'net_income_lag1', 'operating_expenses_lag1', 'operating_income_lag1', 'rd_expense_lag1', 'revenue_lag1', 'stockholders_equity_lag1', 'total_assets_lag1', 'total_liabilities_lag1', 'current_assets_lag1', 'current_liabilities_lag1', 'retained_earnings_lag1', 'cash_lag2', 'cost_of_revenue_lag2', 'gross_profit_lag2', 'net_income_lag2', 'operating_expenses_lag2', 'operating_income_lag2', 'rd_ex

,Missing_Count,Missing_Percentage
cash_lag1,12516,89.810563
revenue_lag1,10570,75.846728
cost_of_revenue_lag1,9015,64.688576
gross_profit_lag1,6111,43.850459
operating_expenses_lag1,5757,41.310276
rd_expense_lag1,4227,30.331515
total_liabilities_lag1,2788,20.005741
current_assets_lag1,2271,16.295924
current_liabilities_lag1,2270,16.288749
operating_income_lag1,1604,11.509759


### Remove Columns with Too Many Missing Values

In [204]:
missing_pct = (df.isnull().sum() / len(df)) * 100

threshold_column = 40
cols_to_drop = sorted(missing_pct[missing_pct > threshold_column].index.tolist())

print(f"Columns with >{threshold_column}% missing values ({len(cols_to_drop)}):")
for col in cols_to_drop:
    print(f"  {col}: {missing_pct[col]:.2f}%")

df_cleaned = df.drop(columns=cols_to_drop)

print(f"\nAfter column filtering:")
print(f"  Original shape: {df.shape}")
print(f"  Cleaned shape: {df_cleaned.shape}")
print(f"  Columns removed: {len(cols_to_drop)}")
print(f"  Columns remaining: {len(df_cleaned.columns)}")

Columns with >40% missing values (20):
  cash_lag1: 89.81%
  cash_lag2: 90.11%
  cash_lag4: 90.57%
  cash_lag5: 90.93%
  cost_of_revenue_lag1: 64.69%
  cost_of_revenue_lag2: 65.52%
  cost_of_revenue_lag4: 67.25%
  cost_of_revenue_lag5: 68.25%
  gross_profit_lag1: 43.85%
  gross_profit_lag2: 44.93%
  gross_profit_lag4: 47.10%
  gross_profit_lag5: 48.62%
  operating_expenses_lag1: 41.31%
  operating_expenses_lag2: 42.58%
  operating_expenses_lag4: 45.16%
  operating_expenses_lag5: 46.70%
  revenue_lag1: 75.85%
  revenue_lag2: 76.07%
  revenue_lag4: 76.23%
  revenue_lag5: 76.27%

After column filtering:
  Original shape: (13936, 79)
  Cleaned shape: (13936, 59)
  Columns removed: 20
  Columns remaining: 59


### Filter Companies by SEC Coverage

In [205]:
base_features = [
    'operating_income',
    'net_income', 
    'current_liabilities',
    'current_assets',
    'stockholders_equity',
    'total_assets'
]

base_lag1_features = [f'{feat}_lag1' for feat in base_features]
base_lag2_features = [f'{feat}_lag2' for feat in base_features]
base_lag4_features = [f'{feat}_lag4' for feat in base_features]
base_lag5_features = [f'{feat}_lag5' for feat in base_features]
all_base_features = base_lag1_features + base_lag2_features + base_lag4_features + base_lag5_features

print(f"Base features: {base_features}")
print(f"Total features for company coverage check: {len(all_base_features)}")
print(f"  - Lag1: {len(base_lag1_features)}")
print(f"  - Lag2: {len(base_lag2_features)}")
print(f"  - Lag4: {len(base_lag4_features)}")
print(f"  - Lag5: {len(base_lag5_features)}")

print("\nChecking feature coverage by company...")
company_coverage = df_cleaned.groupby('Company')[all_base_features].apply(
    lambda x: x.notna().mean().mean()
)

print(f"\nCoverage distribution:")
print(f"  Mean: {company_coverage.mean():.2%}")
print(f"  Median: {company_coverage.median():.2%}")
print(f"  Min: {company_coverage.min():.2%}")
print(f"  Max: {company_coverage.max():.2%}")

print(f"\nCompanies by coverage threshold:")
for threshold in [0, 0.3, 0.5, 0.7, 0.9]:
    count = (company_coverage > threshold).sum()
    print(f"  >{threshold*100:.0f}%: {count} companies")

threshold = 0.8
companies_with_data = company_coverage[company_coverage > threshold].index.tolist()
companies_without_data = company_coverage[company_coverage <= threshold].index.tolist()

print(f"\n=== Applying {threshold*100:.0f}% threshold ===")
print(f"Companies WITH sufficient SEC data: {len(companies_with_data)}")
print(f"Companies WITHOUT sufficient SEC data: {len(companies_without_data)}")

print(f"\nSample companies WITHOUT SEC data:")
print(companies_without_data[:10])

df_filtered = df_cleaned[df_cleaned['Company'].isin(companies_with_data)].copy()

print(f"\nFiltered dataset:")
print(f"  Original shape: {df_cleaned.shape}")
print(f"  Filtered shape: {df_filtered.shape}")
print(f"  Companies: {df_filtered['Company'].nunique()}")
print(f"  Layoff events: {df_filtered['Layoff_Event_Binary'].sum():.0f}")
print(f"  Class balance: {df_filtered['Layoff_Event_Binary'].mean():.4f}")

print(f"\nSample companies kept:")
print(sorted(companies_with_data)[:10])

print(f"\nMissing values after filtering:")
missing = df_filtered.isnull().sum()
missing_pct = (missing / len(df_filtered)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing[missing > 0],
    'Missing_Percentage': missing_pct[missing > 0]
}).sort_values('Missing_Count', ascending=False)
missing_df_filtered = missing_df[~missing_df.index.str.contains('_lag4|_lag5', regex=True)]

display(missing_df_filtered)

Base features: ['operating_income', 'net_income', 'current_liabilities', 'current_assets', 'stockholders_equity', 'total_assets']
Total features for company coverage check: 24
  - Lag1: 6
  - Lag2: 6
  - Lag4: 6
  - Lag5: 6

Checking feature coverage by company...

Coverage distribution:
  Mean: 84.51%
  Median: 95.19%
  Min: 32.69%
  Max: 100.00%

Companies by coverage threshold:
  >0%: 268 companies
  >30%: 268 companies
  >50%: 253 companies
  >70%: 194 companies
  >90%: 145 companies

=== Applying 80% threshold ===
Companies WITH sufficient SEC data: 169
Companies WITHOUT sufficient SEC data: 99

Sample companies WITHOUT SEC data:
['23andme', 'absci', 'aeye', 'affirm', 'airbnb', 'akili interactive', 'akili labs', 'allbirds', 'amplitude', 'appharvest']

Filtered dataset:
  Original shape: (13936, 59)
  Filtered shape: (8788, 59)
  Companies: 169
  Layoff events: 347
  Class balance: 0.0395

Sample companies kept:
['10x genomics', '2u', '8x8', 'absolute software', 'acxiom', 'adaptive

,Missing_Count,Missing_Percentage
rd_expense_lag2,1872,21.301775
rd_expense_lag1,1832,20.846609
total_liabilities_lag2,1392,15.839782
total_liabilities_lag1,1356,15.430132
net_income_lag2,308,3.504779
net_income_lag1,282,3.208921
stockholders_equity_lag2,264,3.004096
stockholders_equity_lag1,251,2.856168
retained_earnings_lag2,185,2.105143
current_liabilities_lag2,172,1.957214


# Feature Engineering

### Create New Features

In [206]:
df_fe = df_filtered.copy()

print("=== FEATURE ENGINEERING ===\n")

print("1. Liquidity Ratios:")
df_fe['current_ratio_lag1'] = df_fe['current_assets_lag1'] / df_fe['current_liabilities_lag1']
print("   - current_ratio_lag1 = current_assets_lag1 / current_liabilities_lag1")

df_fe['working_capital_lag1'] = df_fe['current_assets_lag1'] - df_fe['current_liabilities_lag1']
print("   - working_capital_lag1 = current_assets_lag1 - current_liabilities_lag1")

print("\n2. Profitability Ratios:")
df_fe['roa_lag1'] = df_fe['net_income_lag1'] / df_fe['total_assets_lag1']
print("   - roa_lag1 = net_income_lag1 / total_assets_lag1")

df_fe['roe_lag1'] = df_fe['net_income_lag1'] / df_fe['stockholders_equity_lag1']
print("   - roe_lag1 = net_income_lag1 / stockholders_equity_lag1")

print("\n3. Leverage Ratios:")
df_fe['debt_to_assets_lag1'] = df_fe['total_liabilities_lag1'] / df_fe['total_assets_lag1']
print("   - debt_to_assets_lag1 = total_liabilities_lag1 / total_assets_lag1")

df_fe['debt_to_equity_lag1'] = df_fe['total_liabilities_lag1'] / df_fe['stockholders_equity_lag1']
print("   - debt_to_equity_lag1 = total_liabilities_lag1 / stockholders_equity_lag1")

print("\n4. Efficiency Ratios:")
df_fe['rd_to_assets_lag1'] = df_fe['rd_expense_lag1'] / df_fe['total_assets_lag1']
print("   - rd_to_assets_lag1 = rd_expense_lag1 / total_assets_lag1")

print("\n5. YoY Growth Indicators (with epsilon for stability):")
financial_metrics = ['net_income', 'total_assets', 'operating_income', 
                     'stockholders_equity', 'current_assets', 'current_liabilities']

epsilon = 1e-6

for metric in financial_metrics:
    lag1_col = f'{metric}_lag1'
    lag4_col = f'{metric}_lag4'
    
    if lag1_col in df_fe.columns and lag4_col in df_fe.columns:
        df_fe[f'{metric}_growth_yoy'] = ((df_fe[lag1_col] - df_fe[lag4_col]) / (df_fe[lag4_col].abs() + epsilon)) * 100
        print(f"   - {metric}_growth_yoy = ((lag1 - lag4) / (|lag4| + ε)) × 100")

print("\n6. Quarter-over-Quarter Changes (with epsilon, using existing lag2):")
for metric in ['net_income', 'operating_income']:
    lag1_col = f'{metric}_lag1'
    lag2_col = f'{metric}_lag2'
    
    if lag1_col in df_fe.columns and lag2_col in df_fe.columns:
        df_fe[f'{metric}_change_qoq'] = ((df_fe[lag1_col] - df_fe[lag2_col]) / (df_fe[lag2_col].abs() + epsilon)) * 100
        print(f"   - {metric}_change_qoq = ((lag1 - lag2) / (|lag2| + ε)) × 100")

print("\n7. Financial Distress Indicators:")
df_fe['negative_equity_lag1'] = (df_fe['stockholders_equity_lag1'] < 0).astype(int)
print("   - negative_equity_lag1 = 1 if stockholders_equity_lag1 < 0, else 0")

df_fe['negative_income_lag1'] = (df_fe['net_income_lag1'] < 0).astype(int)
print("   - negative_income_lag1 = 1 if net_income_lag1 < 0, else 0")

df_fe['negative_operating_income_lag1'] = (df_fe['operating_income_lag1'] < 0).astype(int)
print("   - negative_operating_income_lag1 = 1 if operating_income_lag1 < 0, else 0")

df_fe['declining_income_yoy'] = (df_fe['net_income_growth_yoy'] < 0).astype(int)
print("   - declining_income_yoy = 1 if net_income_growth_yoy < 0, else 0")

df_fe['declining_operating_income_yoy'] = (df_fe['operating_income_growth_yoy'] < 0).astype(int)
print("   - declining_operating_income_yoy = 1 if operating_income_growth_yoy < 0, else 0")

print("\n8. Ratio Changes YoY (using lag5 data):")
if 'net_income_lag5' in df_fe.columns and 'total_assets_lag5' in df_fe.columns:
    df_fe['roa_lag5'] = df_fe['net_income_lag5'] / df_fe['total_assets_lag5']
    df_fe['roa_change_yoy'] = df_fe['roa_lag1'] - df_fe['roa_lag5']
    print("   - roa_lag5 = net_income_lag5 / total_assets_lag5")
    print("   - roa_change_yoy = roa_lag1 - roa_lag5")
else:
    print("   - Skipping roa_change_yoy (lag5 data not available)")

if 'total_liabilities_lag5' in df_fe.columns and 'total_assets_lag5' in df_fe.columns:
    df_fe['debt_to_assets_lag5'] = df_fe['total_liabilities_lag5'] / df_fe['total_assets_lag5']
    df_fe['debt_to_assets_change_yoy'] = df_fe['debt_to_assets_lag1'] - df_fe['debt_to_assets_lag5']
    print("   - debt_to_assets_lag5 = total_liabilities_lag5 / total_assets_lag5")
    print("   - debt_to_assets_change_yoy = debt_to_assets_lag1 - debt_to_assets_lag5")
else:
    print("   - Skipping debt_to_assets_change_yoy (lag5 data not available)")

print("\n9. Economic Interaction Features:")
df_fe['income_growth_gdp_interaction'] = df_fe['net_income_growth_yoy'] * df_fe['gdp_growth_rate_lag1']
print("   - income_growth_gdp_interaction = net_income_growth_yoy × gdp_growth_rate_lag1")

df_fe['unemployment_income_interaction'] = df_fe['unemployment_rate_lag1'] * df_fe['net_income_growth_yoy']
print("   - unemployment_income_interaction = unemployment_rate_lag1 × net_income_growth_yoy")

df_fe['operating_income_gdp_interaction'] = df_fe['operating_income_growth_yoy'] * df_fe['gdp_growth_rate_lag1']
print("   - operating_income_gdp_interaction = operating_income_growth_yoy × gdp_growth_rate_lag1")

print("\n10. Lagged Layoff Event:")
df_fe['layoff_event_lag1'] = df_fe.groupby('Company')['Layoff_Event_Binary'].shift(1)
print("   - layoff_event_lag1 = Layoff_Event_Binary shifted by 1 month")

print("\n11. Months Since Last Layoff:")
df_fe = df_fe.sort_values(['Company', 'Date'])

df_fe['last_layoff_date'] = df_fe['Date'].where(df_fe['Layoff_Event_Binary'] == 1, pd.NaT)

df_fe['last_layoff_date'] = df_fe.groupby('Company')['last_layoff_date'].ffill()

df_fe['months_since_last_layoff'] = (df_fe['Date'] - df_fe['last_layoff_date']).dt.days / 30.44

df_fe['months_since_last_layoff'] = df_fe.groupby('Company')['months_since_last_layoff'].shift(1)

df_fe['months_since_last_layoff'] = df_fe['months_since_last_layoff'].fillna(999)

df_fe = df_fe.drop(columns=['last_layoff_date'])

print("   - months_since_last_layoff = (Date - last_layoff_date) / 30.44, then lagged by 1 month")
print("   - Uses only information available BEFORE current month (no leakage)")
print("   - Fill NaN with 999 for companies with no prior layoffs")

new_features = [col for col in df_fe.columns if col not in df_filtered.columns]
print(f"\n=== SUMMARY ===")
print(f"Total new features created: {len(new_features)}")

print(f"\nFeature categories:")
print(f"  - Ratios: {len([f for f in new_features if 'ratio' in f or '_to_' in f or 'roa' in f or 'roe' in f])}")
print(f"  - Growth (YoY): {len([f for f in new_features if 'growth_yoy' in f])}")
print(f"  - Growth (QoQ): {len([f for f in new_features if 'change_qoq' in f])}")
print(f"  - Distress flags: {len([f for f in new_features if 'negative' in f or 'declining' in f])}")
print(f"  - Temporal: {len([f for f in new_features if 'lag' in f or 'months_since' in f])}")

print(f"\nMissing values in new features (top 10):")
new_features_missing = df_fe[new_features].isnull().sum()
missing_pct = (new_features_missing / len(df_fe)) * 100
missing_df = pd.DataFrame({'Count': new_features_missing, 'Percentage': missing_pct})
display(missing_df[missing_df['Count'] > 0].sort_values('Count', ascending=False).head(10))

df_fe.to_csv('data/layoffs_feature_engineered.csv', index=False)
print(f"\nSaved to: data/layoffs_feature_engineered.csv")

=== FEATURE ENGINEERING ===

1. Liquidity Ratios:
   - current_ratio_lag1 = current_assets_lag1 / current_liabilities_lag1
   - working_capital_lag1 = current_assets_lag1 - current_liabilities_lag1

2. Profitability Ratios:
   - roa_lag1 = net_income_lag1 / total_assets_lag1
   - roe_lag1 = net_income_lag1 / stockholders_equity_lag1

3. Leverage Ratios:
   - debt_to_assets_lag1 = total_liabilities_lag1 / total_assets_lag1
   - debt_to_equity_lag1 = total_liabilities_lag1 / stockholders_equity_lag1

4. Efficiency Ratios:
   - rd_to_assets_lag1 = rd_expense_lag1 / total_assets_lag1

5. YoY Growth Indicators (with epsilon for stability):
   - net_income_growth_yoy = ((lag1 - lag4) / (|lag4| + ε)) × 100
   - total_assets_growth_yoy = ((lag1 - lag4) / (|lag4| + ε)) × 100
   - operating_income_growth_yoy = ((lag1 - lag4) / (|lag4| + ε)) × 100
   - stockholders_equity_growth_yoy = ((lag1 - lag4) / (|lag4| + ε)) × 100
   - current_assets_growth_yoy = ((lag1 - lag4) / (|lag4| + ε)) × 100
   - c

,Count,Percentage
rd_to_assets_lag1,1892,21.529358
debt_to_assets_change_yoy,1655,18.832499
debt_to_assets_lag5,1593,18.126991
debt_to_equity_lag1,1463,16.647701
debt_to_assets_lag1,1356,15.430132
roa_change_yoy,740,8.420574
roa_lag5,690,7.851616
roe_lag1,495,5.632681
income_growth_gdp_interaction,459,5.223031
net_income_growth_yoy,459,5.223031



Saved to: data/layoffs_feature_engineered.csv


### Remove Building Blocks and Redundant Columns

In [207]:
import pandas as pd

df_fe = pd.read_csv('data/layoffs_feature_engineered.csv')
df_fe['Date'] = pd.to_datetime(df_fe['Date'])

print(f"Original shape: {df_fe.shape}")
print(f"Original columns: {len(df_fe.columns)}")

# Columns to DROP
columns_to_drop = [
    # 1. Raw building blocks (lag1) - used to create ratios
    'net_income_lag1',
    'operating_income_lag1', 
    'rd_expense_lag1',
    'stockholders_equity_lag1',
    'total_assets_lag1',
    'total_liabilities_lag1',
    'current_assets_lag1',
    'current_liabilities_lag1',
    'retained_earnings_lag1',
    
    # 2. All lag2 columns - only used for QoQ calculation
    'net_income_lag2',
    'operating_income_lag2',
    'rd_expense_lag2',
    'stockholders_equity_lag2',
    'total_assets_lag2',
    'total_liabilities_lag2',
    'current_assets_lag2',
    'current_liabilities_lag2',
    'retained_earnings_lag2',
    
    # 3. All lag4 columns - only used for YoY growth calculation
    'net_income_lag4',
    'operating_income_lag4',
    'rd_expense_lag4',
    'stockholders_equity_lag4',
    'total_assets_lag4',
    'total_liabilities_lag4',
    'current_assets_lag4',
    'current_liabilities_lag4',
    'retained_earnings_lag4',
    
    # 4. All lag5 columns - only used for ratio change calculation
    'net_income_lag5',
    'operating_income_lag5',
    'rd_expense_lag5',
    'stockholders_equity_lag5',
    'total_assets_lag5',
    'total_liabilities_lag5',
    'current_assets_lag5',
    'current_liabilities_lag5',
    'retained_earnings_lag5',
    
    # 5. Intermediate ratio calculations (used only to create change_yoy features)
    'roa_lag5',
    'debt_to_assets_lag5',
    
    # 6. Redundant ratios
    'working_capital_lag1',
    
    # 7. Target leakage columns
    'Layoff_Event_Count',
    'Total_Laid_Off_Sum',
    'Avg_Layoff_Percentage',
    
    # 8. Administrative columns
    'company_sec',
    'fiscal_year',
    'fiscal_period',
    'form',
    
    # 9. Raw CPI
    'cpi_lag1'
]

columns_to_drop_existing = [col for col in columns_to_drop if col in df_fe.columns]

df_cleaned = df_fe.drop(columns=columns_to_drop_existing)

print(f"\nCleaned shape: {df_cleaned.shape}")
print(f"Cleaned columns: {len(df_cleaned.columns)}")
print(f"Columns removed: {len(columns_to_drop_existing)}")

print(f"\n=== FINAL FEATURE SET ===")
print(f"\nPrimary Identifiers (6 columns):")
identifiers = ['Company', 'Date', 'Latest_Industry', 'Latest_Country', 'Latest_Stage', 'Latest_Funds_Raised']
print(identifiers)

print(f"\nTarget Variable (1 column):")
print(['Layoff_Event_Binary'])

print(f"\nMacro/Market Features (Lagged):")
macro_features = [col for col in df_cleaned.columns if col in [
    'unemployment_rate_lag1', 'consumer_confidence_lag1', 'gdp_growth_rate_lag1',
    'fed_funds_rate_lag1', 'avg_jobless_claims_lag1', 'sp500_index_lag1',
    'inflation_rate_yoy_lag1', 'sp500_change_6mo_lag1'
]]
print(f"  Count: {len(macro_features)}")
print(f"  {macro_features}")

print(f"\nFinal Ratios/Growth Features:")
ratio_growth_features = [col for col in df_cleaned.columns if any(x in col for x in [
    'ratio', 'roa', 'roe', 'debt_to', 'rd_to', 'growth_yoy', 'change_qoq', 'change_yoy'
]) and not any(x in col for x in ['lag5'])]
print(f"  Count: {len(ratio_growth_features)}")
print(f"  {ratio_growth_features}")

print(f"\nBinary/Temporal Features:")
binary_temporal = [col for col in df_cleaned.columns if any(x in col for x in [
    'negative', 'declining', 'layoff_event_lag1', 'months_since'
])]
print(f"  Count: {len(binary_temporal)}")
print(f"  {binary_temporal}")

print(f"\nInteraction Features:")
interaction = [col for col in df_cleaned.columns if 'interaction' in col]
print(f"  Count: {len(interaction)}")
print(f"  {interaction}")

print(f"\n=== SUMMARY ===")
print(f"Total features: {len(df_cleaned.columns)}")
print(f"  - Identifiers: {len(identifiers)}")
print(f"  - Target: 1")
print(f"  - Macro/Market: {len(macro_features)}")
print(f"  - Ratios/Growth: {len(ratio_growth_features)}")
print(f"  - Binary/Temporal: {len(binary_temporal)}")
print(f"  - Interactions: {len(interaction)}")

df_cleaned.to_csv('data/layoffs_features_cleaned.csv', index=False)
print(f"\nSaved final modeling dataset to: data/layoffs_features_cleaned.csv")

print(f"\nFinal columns:")
print(df_cleaned.columns.tolist())

Original shape: (8788, 88)
Original columns: 88

Cleaned shape: (8788, 41)
Cleaned columns: 41
Columns removed: 47

=== FINAL FEATURE SET ===

Primary Identifiers (6 columns):
['Company', 'Date', 'Latest_Industry', 'Latest_Country', 'Latest_Stage', 'Latest_Funds_Raised']

Target Variable (1 column):
['Layoff_Event_Binary']

Macro/Market Features (Lagged):
  Count: 8
  ['unemployment_rate_lag1', 'consumer_confidence_lag1', 'gdp_growth_rate_lag1', 'fed_funds_rate_lag1', 'avg_jobless_claims_lag1', 'sp500_index_lag1', 'inflation_rate_yoy_lag1', 'sp500_change_6mo_lag1']

Final Ratios/Growth Features:
  Count: 16
  ['current_ratio_lag1', 'roa_lag1', 'roe_lag1', 'debt_to_assets_lag1', 'debt_to_equity_lag1', 'rd_to_assets_lag1', 'net_income_growth_yoy', 'total_assets_growth_yoy', 'operating_income_growth_yoy', 'stockholders_equity_growth_yoy', 'current_assets_growth_yoy', 'current_liabilities_growth_yoy', 'net_income_change_qoq', 'operating_income_change_qoq', 'roa_change_yoy', 'debt_to_assets

### Impute Missing data

In [208]:
import pandas as pd
import numpy as np

df_cleaned = pd.read_csv('data/layoffs_features_cleaned.csv')
df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'])

print(f"Dataset shape: {df_cleaned.shape}")
print(f"Companies: {df_cleaned['Company'].nunique()}")

print(f"\n=== Missing Values Before Imputation ===")
missing = df_cleaned.isnull().sum()
missing_pct = (missing / len(df_cleaned)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing[missing > 0],
    'Missing_Percentage': missing_pct[missing > 0]
}).sort_values('Missing_Count', ascending=False)
display(missing_df)

# Identify columns to impute (exclude identifiers and target)
cols_to_exclude = ['Company', 'Date', 'Latest_Industry', 'Latest_Country', 
                   'Latest_Stage', 'Latest_Funds_Raised', 'Layoff_Event_Binary']

cols_to_impute = [col for col in df_cleaned.columns if col not in cols_to_exclude and df_cleaned[col].isnull().sum() > 0]

print(f"\n=== Imputation Strategy ===")
print(f"Columns to impute: {len(cols_to_impute)}")
print(f"Method: Industry Median (group by Latest_Industry)")

df_imputed = df_cleaned.copy()

for col in cols_to_impute:
    print(f"\nImputing {col}...")
    missing_before = df_imputed[col].isnull().sum()
    
    # Step 1: Industry median imputation
    df_imputed[col] = df_imputed.groupby('Latest_Industry')[col].transform(
        lambda x: x.fillna(x.median())
    )
    
    # Step 2: Global median fallback for remaining NaNs
    remaining_missing = df_imputed[col].isnull().sum()
    if remaining_missing > 0:
        global_median = df_imputed[col].median()
        df_imputed[col] = df_imputed[col].fillna(global_median)
        print(f"  Industry median: {missing_before - remaining_missing} values")
        print(f"  Global median fallback: {remaining_missing} values")
    else:
        print(f"  Industry median: {missing_before} values")

print(f"\n=== Missing Values After Imputation ===")
missing_after = df_imputed.isnull().sum()
missing_after_pct = (missing_after / len(df_imputed)) * 100
missing_after_df = pd.DataFrame({
    'Missing_Count': missing_after[missing_after > 0],
    'Missing_Percentage': missing_after_pct[missing_after > 0]
}).sort_values('Missing_Count', ascending=False)

if len(missing_after_df) > 0:
    display(missing_after_df)
else:
    print("No missing values remaining!")

print(f"\n=== Imputation Summary ===")
print(f"Total columns imputed: {len(cols_to_impute)}")
print(f"Total missing values before: {missing.sum()}")
print(f"Total missing values after: {missing_after.sum()}")
print(f"Imputation success rate: {(1 - missing_after.sum() / missing.sum()) * 100:.2f}%")

df_imputed.to_csv('data/layoffs_imputed.csv', index=False)
print(f"\nSaved imputed dataset to: data/layoffs_imputed.csv")

Dataset shape: (8788, 41)
Companies: 169

=== Missing Values Before Imputation ===


,Missing_Count,Missing_Percentage
rd_to_assets_lag1,1892,21.529358
debt_to_assets_change_yoy,1655,18.832499
debt_to_equity_lag1,1463,16.647701
debt_to_assets_lag1,1356,15.430132
roa_change_yoy,740,8.420574
roe_lag1,495,5.632681
unemployment_income_interaction,459,5.223031
net_income_growth_yoy,459,5.223031
income_growth_gdp_interaction,459,5.223031
stockholders_equity_growth_yoy,379,4.312699



=== Imputation Strategy ===
Columns to impute: 20
Method: Industry Median (group by Latest_Industry)

Imputing current_ratio_lag1...
  Industry median: 129 values

Imputing roa_lag1...
  Industry median: 357 values

Imputing roe_lag1...
  Industry median: 495 values

Imputing debt_to_assets_lag1...
  Industry median: 1304 values
  Global median fallback: 52 values

Imputing debt_to_equity_lag1...
  Industry median: 1411 values
  Global median fallback: 52 values

Imputing rd_to_assets_lag1...
  Industry median: 1684 values
  Global median fallback: 208 values

Imputing net_income_growth_yoy...
  Industry median: 459 values

Imputing total_assets_growth_yoy...
  Industry median: 315 values

Imputing operating_income_growth_yoy...
  Industry median: 292 values

Imputing stockholders_equity_growth_yoy...
  Industry median: 379 values

Imputing current_assets_growth_yoy...
  Industry median: 330 values

Imputing current_liabilities_growth_yoy...
  Industry median: 345 values

Imputing net

c:\Users\zhang\miniconda3\envs\7374_A2\lib\site-packages\numpy\lib\nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\zhang\miniconda3\envs\7374_A2\lib\site-packages\numpy\lib\nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\zhang\miniconda3\envs\7374_A2\lib\site-packages\numpy\lib\nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\zhang\miniconda3\envs\7374_A2\lib\site-packages\numpy\lib\nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\zhang\miniconda3\envs\7374_A2\lib\site-packages\numpy\lib\nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


  Industry median: 459 values

Imputing operating_income_gdp_interaction...
  Industry median: 292 values

Imputing layoff_event_lag1...
  Industry median: 169 values

=== Missing Values After Imputation ===
No missing values remaining!

=== Imputation Summary ===
Total columns imputed: 20
Total missing values before: 12159
Total missing values after: 0
Imputation success rate: 100.00%

Saved imputed dataset to: data/layoffs_imputed.csv


### Verify Imputation Results

In [209]:
print(f"Final dataset shape: {df_imputed.shape}")
print(f"Companies: {df_imputed['Company'].nunique()}")
print(f"Date range: {df_imputed['Date'].min().date()} to {df_imputed['Date'].max().date()}")

print(f"\n=== Target Distribution ===")
print(df_imputed['Layoff_Event_Binary'].value_counts())
print(f"Class balance: {df_imputed['Layoff_Event_Binary'].mean():.4f}")

print(f"\n=== Feature Summary ===")
feature_cols = [col for col in df_imputed.columns if col not in 
                ['Company', 'Date', 'Latest_Industry', 'Latest_Country', 
                 'Latest_Stage', 'Latest_Funds_Raised', 'Layoff_Event_Binary']]

print(f"Total features for modeling: {len(feature_cols)}")

print(f"\n=== Sample Statistics ===")
print("Example: roa_lag1 after imputation")
print(df_imputed['roa_lag1'].describe())


Final dataset shape: (8788, 41)
Companies: 169
Date range: 2020-03-01 to 2024-06-01

=== Target Distribution ===
0.0    8441
1.0     347
Name: Layoff_Event_Binary, dtype: int64
Class balance: 0.0395

=== Feature Summary ===
Total features for modeling: 34

=== Sample Statistics ===
Example: roa_lag1 after imputation
count    8.788000e+03
mean    -8.446016e+03
std      4.400951e+05
min     -2.380400e+07
25%     -9.666703e-02
50%     -1.720054e-02
75%      2.659123e-02
max      3.332031e+00
Name: roa_lag1, dtype: float64


### Check Categorical Features

In [210]:
df_imputed = pd.read_csv('data/layoffs_imputed.csv')
df_imputed['Date'] = pd.to_datetime(df_imputed['Date'])

categorical_cols = ['Latest_Industry', 'Latest_Country', 'Latest_Stage']

print("Categorical feature analysis:\n")
for col in categorical_cols:
    print(f"{col}:")
    print(f"  Unique values: {df_imputed[col].nunique()}")
    print(f"  Top 5 categories:\n{df_imputed[col].value_counts().head()}")
    print()

Categorical feature analysis:

Latest_Industry:
  Unique values: 25
  Top 5 categories:
Healthcare        1248
Other              919
Consumer           854
Transportation     834
Security           624
Name: Latest_Industry, dtype: int64

Latest_Country:
  Unique values: 11
  Top 5 categories:
United States    8326
Canada            104
France            104
Israel             54
Poland             52
Name: Latest_Country, dtype: int64

Latest_Stage:
  Unique values: 7
  Top 5 categories:
Post-IPO          8515
Series D            93
Acquired            49
Private Equity      41
Series H            37
Name: Latest_Stage, dtype: int64



### One-Hot Encode Industry and Stage

In [211]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

industry_counts = df_imputed['Latest_Industry'].value_counts()
top_industries = industry_counts.head(15).index.tolist()

df_imputed['Latest_Industry_Grouped'] = df_imputed['Latest_Industry'].apply(
    lambda x: x if x in top_industries else 'Other'
)

print(f"Industry grouping:")
print(f"  Original categories: {df_imputed['Latest_Industry'].nunique()}")
print(f"  After grouping: {df_imputed['Latest_Industry_Grouped'].nunique()}")
print(f"\nGrouped distribution:\n{df_imputed['Latest_Industry_Grouped'].value_counts()}")

industry_dummies = pd.get_dummies(df_imputed['Latest_Industry_Grouped'], 
                                  prefix='Industry', drop_first=True)

stage_dummies = pd.get_dummies(df_imputed['Latest_Stage'], 
                               prefix='Stage', drop_first=True)

print(f"\nOne-hot encoded features:")
print(f"  Industry features: {industry_dummies.shape[1]}")
print(f"  Stage features: {stage_dummies.shape[1]}")
print(f"  Total new features: {industry_dummies.shape[1] + stage_dummies.shape[1]}")

Industry grouping:
  Original categories: 25
  After grouping: 15

Grouped distribution:
Other             1872
Healthcare        1248
Consumer           854
Transportation     834
Security           624
Hardware           575
Retail             545
Marketing          520
Data               416
Finance            416
Real Estate        208
Media              208
Manufacturing      156
Food               156
Infrastructure     156
Name: Latest_Industry_Grouped, dtype: int64

One-hot encoded features:
  Industry features: 14
  Stage features: 6
  Total new features: 20


### Log Transform Funds Raised

In [212]:
df_imputed['Latest_Funds_Raised_Log'] = np.log1p(df_imputed['Latest_Funds_Raised'])

print("Log transformation of Latest_Funds_Raised:")
print(f"\nOriginal (Latest_Funds_Raised):")
print(df_imputed['Latest_Funds_Raised'].describe())
print(f"\nLog transformed (Latest_Funds_Raised_Log):")
print(df_imputed['Latest_Funds_Raised_Log'].describe())

Log transformation of Latest_Funds_Raised:

Original (Latest_Funds_Raised):
count      8788.000000
mean        916.638507
std        7199.910378
min           0.000000
25%           0.000000
50%           0.000000
75%         173.000000
max      121900.000000
Name: Latest_Funds_Raised, dtype: float64

Log transformed (Latest_Funds_Raised_Log):
count    8788.000000
mean        2.192082
std         3.035803
min         0.000000
25%         0.000000
50%         0.000000
75%         5.159055
max        11.710965
Name: Latest_Funds_Raised_Log, dtype: float64


### Clip All Ratio and Growth Features

In [213]:
features_to_clip = [
    # 1. High-Volatility Ratios (Leverage and Profitability)
    'roe_lag1', 
    'debt_to_equity_lag1',
    'current_ratio_lag1',
    'roa_lag1', 
    'debt_to_assets_lag1',
    'rd_to_assets_lag1',

    # 2. Financial Momentum/Growth Metrics (YoY and QoQ Change)
    'net_income_growth_yoy', 
    'total_assets_growth_yoy',
    'operating_income_growth_yoy',
    'stockholders_equity_growth_yoy',
    'current_assets_growth_yoy',
    'current_liabilities_growth_yoy',
    'net_income_change_qoq',
    'operating_income_change_qoq',

    # 3. Ratio Change Metrics
    'roa_change_yoy',
    'debt_to_assets_change_yoy',

    # 4. Interaction Terms
    'income_growth_gdp_interaction', 
    'unemployment_income_interaction',
    'operating_income_gdp_interaction'
]

print(f"Clipping {len(features_to_clip)} ratio/growth features:\n")
for col in features_to_clip:
    p1 = df_imputed[col].quantile(0.01)
    p99 = df_imputed[col].quantile(0.99)
    
    before_min = df_imputed[col].min()
    before_max = df_imputed[col].max()
    
    df_imputed[col] = df_imputed[col].clip(lower=p1, upper=p99)
    
    print(f"{col}: [{before_min:.2f}, {before_max:.2f}] -> [{p1:.2f}, {p99:.2f}]")

print(f"\nClipping complete")

Clipping 19 ratio/growth features:

roe_lag1: [-23804000.00, 46.15] -> [-4.57, 3.26]
debt_to_equity_lag1: [-869.72, 598.32] -> [-27.06, 26.90]
current_ratio_lag1: [0.00, 85.59] -> [0.25, 18.69]
roa_lag1: [-23804000.00, 3.33] -> [-0.92, 0.23]
debt_to_assets_lag1: [0.00, 12.32] -> [0.04, 1.41]
rd_to_assets_lag1: [0.00, 144.12] -> [0.01, 0.38]
net_income_growth_yoy: [-422804.25, 113306.31] -> [-5222.57, 2024.88]
total_assets_growth_yoy: [-99.63, 210879389020.61] -> [-44.32, 342.21]
operating_income_growth_yoy: [-240089.51, 2961000.00] -> [-5456.93, 1284.47]
stockholders_equity_growth_yoy: [-4334.59, 855600000000000.00] -> [-312.08, 4716.01]
current_assets_growth_yoy: [-98.70, 1546303.94] -> [-62.43, 3667.23]
current_liabilities_growth_yoy: [-94.43, 33684.47] -> [-57.66, 485.11]
net_income_change_qoq: [-96098.81, 122638.20] -> [-1138.90, 1127.79]
operating_income_change_qoq: [-19804.23, 173700.00] -> [-1021.74, 534.06]
roa_change_yoy: [-8.82, 23803999.99] -> [-0.62, 0.45]
debt_to_assets_ch

### Combine All Features

In [214]:
exclude_cols = ['Company', 'Date', 'Latest_Industry', 'Latest_Country', 
                'Latest_Stage', 'Latest_Funds_Raised', 'Layoff_Event_Binary',
                'Latest_Industry_Grouped']

numeric_features = [col for col in df_imputed.columns if col not in exclude_cols]

df_final = pd.concat([
    df_imputed[numeric_features],
    industry_dummies,
    stage_dummies
], axis=1)

df_final['Date'] = df_imputed['Date']
df_final['Company'] = df_imputed['Company']
df_final['Latest_Country'] = df_imputed['Latest_Country']
df_final['Layoff_Event_Binary'] = df_imputed['Layoff_Event_Binary']

print(f"Final feature matrix:")
print(f"  Numeric features: {len(numeric_features)}")
print(f"  Industry dummies: {industry_dummies.shape[1]}")
print(f"  Stage dummies: {stage_dummies.shape[1]}")
print(f"  Total features: {df_final.shape[1] - 4}")
print(f"\nFinal shape: {df_final.shape}")
print(f"Missing values: {df_final.isnull().sum().sum()}")

Final feature matrix:
  Numeric features: 35
  Industry dummies: 14
  Stage dummies: 6
  Total features: 55

Final shape: (8788, 59)
Missing values: 0


### Save Preprocessed Dataset

In [215]:
df_final.to_csv('data/layoffs_modeling_ready.csv', index=False)
print(f"Saved preprocessed dataset to: data/layoffs_modeling_ready.csv")
print(f"Shape: {df_final.shape}")
print(f"Features for modeling: {df_final.shape[1] - 4}")

Saved preprocessed dataset to: data/layoffs_modeling_ready.csv
Shape: (8788, 59)
Features for modeling: 55
